# 1. 関数の特殊属性

In [1]:
from typing import List

In [9]:
def hoge(p1: int, p2: "Hoge") -> List[int]:
    """
    hogehoge yana
    """
    pass

def hoge2(p1, p2):
    # type: (int, "Hoge") -> List[int]
    pass

## docstring

In [10]:
hoge.__doc__

'\n    hogehoge yana\n    '

## アノテーション

In [12]:
hoge.__annotations__

{'p1': int, 'p2': 'Hoge', 'return': typing.List[int]}

In [13]:
hoge2.__annotations__

{}

##  `__dict__`

In [14]:
# 任意の関数属性をサポートするための名前空間が収められています。
hoge.__dict__

{}

In [15]:
hoge.fuga = 56

In [16]:
hoge.__dict__

{'fuga': 56}

# 2. メタクラス

クラスオブジェクト生成へ介入

の前に前提知識

## 用語の整理

In [84]:
# クラス定義
class Container:
    def __init__(self): self.value = None
    def set_value(self, v): self.value = v

c = Container()
# c: クラスインスタンス
# Container: クラスオブジェクト

## `object.__new__(cls[,...])`

`__init__(self[, ...])` 呼び出し前のインスタンスオブジェクト `self` 生成に介入

In [63]:
class BoxedInt(int):
    def __init__(self, v, container: Container):
        super().__init__(v)
        self.container = container
        self.container.set_value = self

In [64]:
c = Container()
bi = BoxedInt(3, c)
bi

TypeError: 'Container' object cannot be interpreted as an integer

In [65]:
class BoxedInt(int):
    def __new__(cls, v, container: Container):
        instance = super().__new__(cls, v)
        instance.container = container
        instance.container.set_value = instance
        return instance


In [66]:
c = Container()
bi = BoxedInt(3, c)
bi, bi.container, type(bi)

(3, <__main__.Container at 0x2d0e9d21320>, __main__.BoxedInt)

## クラスオブジェクト生成の仕組み

In [82]:
class Clazz(Container):
    green = 56
    def red(self):
        return self.green * 3
z = ClazzZ()
ClazzZ.green, z.red()    

(56, 168)

In [83]:
ClazzZ = type("ClazzZ", (Container,), {"green": 56, "red": lambda self: self.green*3})
z = ClazzZ()
ClazzZ.green, z.red()

(56, 168)

## メタクラス

`type()` の挙動を変えることをできれば、クラスオブジェクト生成へ介入できる

In [89]:
class Meta(type):
    #      ^^^^
    # クラス定義の内容が渡される
    def __new__(mcs, cls_name, cls_bases, cls_dict):
        print('meta', cls_dict)
        return super().__new__(mcs, cls_name, cls_bases, cls_dict)

class A(metaclass=Meta):
    green = 56
    def red(self): return self.green * 3

meta {'__module__': '__main__', '__qualname__': 'A', 'green': 56, 'red': <function A.red at 0x000002D0E9D26F28>}


In [99]:
class NoHoge(type):
    def __new__(mcs, cls_name, cls_bases, raw_cls_dict):
        cls_dict = {}
        for name, v in raw_cls_dict.items():
            if name.startswith("hoge"):
                raise AttributeError(f"{name}: cannot define member which starts with hoge")
            else:
                cls_dict[name] = v
        return super().__new__(mcs, cls_name, cls_bases, cls_dict)

class Hoge(metaclass=NoHoge):
    hoge = 56
    def hogemoge(i):
        return f"static hoge {i}"

AttributeError: hoge: cannot define member which starts with hoge

# 3. ディスクリプタ

## デスクリプタ

[3.3.2.2. デスクリプタ (descriptor) の実装](https://docs.python.jp/3/reference/datamodel.html#implementing-descriptors)

> 以下のメソッドは、このメソッドを持つクラス (いわゆる デスクリプタ(descriptor) クラス) のインスタンスが、 オーナー (owner) クラスに存在するときにのみ適用されます (デスクリプタは、オーナーのクラス辞書か、その親のいずれかのクラス辞書になければなりません)。 以下の例では、"属性" とは、名前がオーナークラスの __dict__ のプロパティ (porperty) のキーであるような属性を指します。

In [112]:
class StaticMethod:
    def __init__(self, f):
        self.f = f
        
    def __get__(self, obj, clazz=None):
        return self.f
    
class Hoge:
    im_static = StaticMethod(lambda : print("no self"))
    
    @StaticMethod
    def im_static_deco():
        print("no sel decof")
        
Hoge.im_static()
Hoge.im_static_deco()

hoge = Hoge()
hoge.im_static()
hoge.im_static_deco()

no self
no sel decof
no self
no sel decof


## `__set_name__`

メタクラスには致命的な欠点があり、ライブラリ/フレームワーク開発においてたいへんほげほげ

[3.3.3.2. 適切なメタクラスの決定](https://docs.python.jp/3/reference/datamodel.html#determining-the-appropriate-metaclass)
> - ベースクラスも明示的なメタクラスも指定されなかった場合、 type() が使われます
- 明示的にメタクラスが指定され、それが type() のインスタンス でない 場合、それが直接メタクラスとして使われます
- type() のインスタンスが明示的にメタクラスとして指定されたり、ベースクラスが定義されている場合、最も派生的なメタクラスが使われます
> 
> 最も派生的なメタクラスは、(もしあれば) 明示的に指定されたメタクラスと、指定されたすべてのベースクラスのメタクラスから選ばれます。最も派生的なメタクラスは、これらのメタクラス候補のすべてのサブタイプであるようなものです。メタクラス候補のどれもその基準を満たさなければ、クラス定義は TypeError で失敗します。

In [100]:
class MetaA(type): pass
class MetaB(type): pass

class Base(metaclass=MetaA): pass
class Concrete(Base, metaclass=MetaB): pass

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

In [101]:
class MetaC(MetaA, MetaB): pass
    
class Concrete(Base, metaclass=MetaC): pass

Python 3.6 からデスクリプタに `__set_name__` が追加され、従来メタクラスを必要としたユースケースのいくらかメタクラスに依存せず簡潔に記述できるようになった

In [116]:
class Trait:
    def __init__(self, minimum, maximum):
        self.minimum = minimum
        self.maximum = maximum

    def __get__(self, instance, owner):
        return instance.__dict__[self.key]

    def __set__(self, instance, value):
        if self.minimum < value < self.maximum:
            instance.__dict__[self.key] = value
        else:
            raise ValueError("value not in range")

    def __set_name__(self, owner, name):
        self.key = name
        
class ScreenPos:
    x = Trait(0, 2560)
    y = Trait(0, 1440)
    
p = ScreenPos()
p.x = 100
p.x
p.y = 1000
p.y
p.x = -1900
p.x

ValueError: value not in range